# Prepare_callbacks

In [1]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\DL_Projects\\2025-projects\\End-to-End-Industrial-Grade-DL-Project\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\HP\\Desktop\\DS\\DL_Projects\\2025-projects\\End-to-End-Industrial-Grade-DL-Project'

In [4]:
#preparing entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallabacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
#preparing configguration manager

from src.CNNClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.CNNClassifier.utils import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    #preparing_callbacks_configurations
    def get_callbacks_config(self) -> CallabacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_ckpt_dir)
            ])

        callbacks_config = CallabacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )
        return callbacks_config

    #prepare_function to crate model_checkpoints and tensorboard_logs


In [6]:
#prepare my components
import os
import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request
import time


class Callbacks:
    def __init__(self, config:CallabacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d:%H-%M-%S")
        tb_runing_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir= tb_runing_log_dir)
    
    @property
    def _create_chkp_callbacks(self):
        tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )

    #define methods to get tensorboard checkpoints
    def get_tb_chkp_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_chkp_callbacks
        ]

In [7]:
#preparing pipeline
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()    
    callbacks = Callbacks(config = callbacks_config)
    callbacks_list = callbacks.get_tb_chkp_callbacks()
except Exception as e:
    raise e



[2025-10-05 19:10:21,454: INFO: common]: yaml file: configs\config.yaml loaded successfully]
[2025-10-05 19:10:21,456: INFO: common]: yaml file: params.yaml loaded successfully]
[2025-10-05 19:10:21,458: INFO: common]: Directory created at: artifacts]
[2025-10-05 19:10:21,460: INFO: common]: Directory created at: artifacts\callbacks\tensorboard_root_log_dir]
[2025-10-05 19:10:21,462: INFO: common]: Directory created at: artifacts\callbacks\checkpoint_dir]
